## Exercício Desafio

- Digamos que seu chefe pediu para você um relatório da análise dos salários da unidade de San Francisco da empresa. O objetivo dele é entender:

1. Qual foi a evolução do salário médio ao longo dos anos (TotalPay e TotalPayBenefits)
2. Quantos funcionários tivemos ao longo dos anos
3. Qual foi a evolução do total gasto com salário ao longo dos anos (TotalPayBenefits)

- Base de Dados a ser usada: salarios.sqlite

### Importação da Base de Dados

In [1]:
import pyodbc
import pandas as pd

dados_conexao = ("Driver={SQLite3 ODBC Driver};"
                 "Server=localhost;"
                 "Database=salarios.sqlite")

conexao = pyodbc.connect(dados_conexao)
cursor = conexao.cursor()

### Análise de Dados

In [2]:
# garantindo que estamos só com San Francisco
cursor.execute('''
SELECT * FROM Salaries WHERE Agency="San Francisco" 
''')
valores = cursor.fetchall()
descricao = cursor.description
colunas = [tupla[0] for tupla in descricao]
# print(valores[:10])
# print(descricao)
cursor.close()
conexao.close()



In [3]:
df_sanfrancisco = pd.DataFrame.from_records(valores, columns = colunas)
display(df_sanfrancisco)
print(df_sanfrancisco.info())

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.00,400184.25,NaN,567595.43,567595.43,2011,,San Francisco,
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,NaN,538909.28,538909.28,2011,,San Francisco,
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.60,NaN,335279.91,335279.91,2011,,San Francisco,
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.00,56120.71,198306.90,NaN,332343.61,332343.61,2011,,San Francisco,
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.60,9737.00,182234.59,NaN,326373.19,326373.19,2011,,San Francisco,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148649,148650,Roy I Tillery,Custodian,0.00,0.00,0.00,0.0,0.00,0.00,2014,,San Francisco,PT
148650,148651,Not provided,Not provided,NaN,NaN,NaN,NaN,0.00,0.00,2014,,San Francisco,
148651,148652,Not provided,Not provided,NaN,NaN,NaN,NaN,0.00,0.00,2014,,San Francisco,
148652,148653,Not provided,Not provided,NaN,NaN,NaN,NaN,0.00,0.00,2014,,San Francisco,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148654 entries, 0 to 148653
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Id                148654 non-null  int64  
 1   EmployeeName      148654 non-null  object 
 2   JobTitle          148654 non-null  object 
 3   BasePay           148045 non-null  float64
 4   OvertimePay       148650 non-null  float64
 5   OtherPay          148650 non-null  float64
 6   Benefits          112491 non-null  float64
 7   TotalPay          148654 non-null  float64
 8   TotalPayBenefits  148654 non-null  float64
 9   Year              148654 non-null  int64  
 10  Notes             148654 non-null  object 
 11  Agency            148654 non-null  object 
 12  Status            148654 non-null  object 
dtypes: float64(6), int64(2), object(5)
memory usage: 14.7+ MB
None


##### 1. Qual foi a evolução do salário médio ao longo dos anos

In [4]:
df_salario = df_sanfrancisco[['Year', 'TotalPay', 'TotalPayBenefits' ]].groupby('Year').mean()
display(df_salario)

,TotalPay,TotalPayBenefits
Year,,
2011,71744.103871,71744.103871
2012,74113.262265,100553.229232
2013,77611.443142,101440.519714
2014,75463.918140,100250.918884


##### 2. Quantos funcionários tivemos ao longo dos anos

In [5]:
df_funcionario = df_sanfrancisco[['Id', 'Year']].groupby('Year').count()
df_funcionario = df_funcionario.rename(columns = {'Id': 'Qtde'})
display(df_funcionario)

,Qtde
Year,
2011,36159
2012,36766
2013,37606
2014,38123


##### 3. Qual foi a evolução do total gasto com salário ao longo dos anos

In [6]:
def formatar(valor):
    return 'R${:,.2f}'.format(valor)
# Separador de milhar, duas casa decimais

df_total = df_sanfrancisco[['TotalPay', 'TotalPayBenefits', 'Year']].groupby('Year').sum()
df_total['TotalPay'] = df_total['TotalPay'].apply(formatar)
df_total['TotalPayBenefits'] = df_total['TotalPayBenefits'].apply(formatar) 
display(df_total)

,TotalPay,TotalPayBenefits
Year,,
2011,"R$2,594,195,051.88","R$2,594,195,051.88"
2012,"R$2,724,848,200.44","R$3,696,940,025.96"
2013,"R$2,918,655,930.80","R$3,814,772,184.37"
2014,"R$2,876,910,951.26","R$3,821,865,780.60"


#### Quantos funcionarios por departamento?

In [7]:
df_qtde_dp = df_sanfrancisco[['Id', 'JobTitle']].groupby('JobTitle').count()
display(df_qtde_dp.head(n=5).sort_values(by=['Id'], ascending=False))


,Id
JobTitle,
ACCOUNT CLERK,83
ACCOUNTANT INTERN,48
ACCOUNTANT,5
"ACPO,JuvP, Juv Prob (SFERS)",1
ACUPUNCTURIST,1
